In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [ ]:
# 출력시 텍스트 제한 해제
pd.set_option('display.max_colwidth', None)

# 출력시 텍스트 제한
# pd.reset_option('display.max_colwidth')

# 모든 행 보기 설정
pd.set_option('display.max_rows', None)

# 모든 행 보기 설정 해제
# pd.set_option('display.max_rows')

In [5]:
mainPaths =[ 
    # contents_1
    './templates/contents/contents_1/귀어귀촌개념.html', 
    './templates/contents/contents_1/귀어귀촌준비절차.html', 
    './templates/contents/contents_1/어업면허허가.html', 
    './templates/contents/contents_1/어업정의.html',

    # contents_2
    './templates/contents/contents_2/귀어정책지원교육.html', 
    './templates/contents/contents_2/양식장및어선임대사업.html', 
    './templates/contents/contents_2/어촌이사혜택.html', 
    './templates/contents/contents_2/청년어촌정착지원금.html', 

    # # contents_3
    './templates/contents/contents_3/유해해양생물_종류.html', 
    './templates/contents/contents_3/유해해양생물_대응법.html', 
    './templates/contents/contents_3/이상생태현상_굴폐사.html', 
    './templates/contents/contents_3/이상생태현상_산호초백화.html', 
    './templates/contents/contents_3/이상생태현상_대응법.html', 

    # # contents_4
    './templates/contents/contents_4/IoTAI활용스마트양식.html', 
    './templates/contents/contents_4/친환경지속가능어업.html', 

    # # contents_5
    './templates/contents/contents_5/어업종류.html', 
    './templates/contents/contents_5/지역별주력어종.html', 
    './templates/contents/contents_5/금어기및금지체장.html',
    './templates/contents/contents_5/낚시금지구역.html',
    './templates/contents/contents_5/종류별낚시설명.html', 
    './templates/contents/contents_5/지역별낚시명소.html', 
    './templates/contents/contents_5/지역별낚시산업.html'
    ]

file_path = mainPaths
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "다.", "요.", ".", "\n", " ", ""]
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(separator=" ",strip=True)
                content = contents[k].get_text(separator=" ", strip=True)
                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(separator=" ", strip=True)
            content = contents[0].get_text(separator=" ", strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print(f"{f}의 제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df
# len(df['title'].unique())


,title,content,folder,page
0,귀어/귀어업,어업활동을 하기 위해 타지에서부터 어촌으로 이사와 거주하는 것 J턴 : 어촌 출신 도시거주자가 연고가 없는 타향 어촌으로 이주 U턴 : 어촌 출신 도시거주자가 고향 어촌으로 다시 복귀 I턴 : 도시거주자가 어촌으로 이주,contents_1,귀어귀촌개념
1,귀촌/귀어촌,귀촌/귀어촌어업활동과 관련없이 타지에서 어촌으로 이주하는 것,contents_1,귀어귀촌개념
2,귀어귀촌 준비절차,"STEP 01. 귀어를 준비하는 사람은 수산 관련기관 홈페이지에서 정보를 수집하고, 전문가와 상담을 통해 기초지식을 쌓는다. STEP 02. 귀어 희망자는 교육기관에서 어업(양식업)과 관련된 이론 및 기술 교육을 받고, 현장체험을 통해 실제 어촌 작업을 경험한다. STEP 03. 개인의 적성, 기술 수준, 자본 등을 고려하여 자신에게 적합한 업종을 신중하게 선택한다. STEP 04. 어업 또는 양식업에 직접 참여하면서 실질적인 기술을 습득한다. STEP 05. 생활 여건과 선택한 업종에 적합한 입지 조건을 바탕으로 정착할 지역을 선정한다. STEP 06. 지자체 또는 선도어가의 조언을 참고하여, 현실적으로 실행 가능한 창업계획을 수립한다. STEP 07. 창업 및 생활에 필요한 자금을 확보하고, 국가의 귀어 지원 제도를 활용한다. STEP 08. 어업 활동에 필요한 어선, 양식장, 주택 등 기반시설을 마련한",contents_1,귀어귀촌준비절차
3,귀어귀촌 준비절차,"다. STEP 08. 어업 활동에 필요한 어선, 양식장, 주택 등 기반시설을 마련한다. STEP 09. 행정 절차를 완료한 후, 수립한 계획에 따라 사업을 본격적으로 추진한다. STEP 10. 마을 행사나 회의, 공동작업 등에 적극 참여하면서 지역 주민과의 융화를 도모한다.",contents_1,귀어귀촌준비절차
4,어업면허,"어업면허 는 특정한 수면을 독점적으로 사용할 수 있도록 국가가 권리를 부여하는 제도입니다. \n 양식어업, 정치어업, 공동어업 등이 이에 해당하며 면허를 받은 자는 지정된 어장에서 타인의 출입을 배제하고 독점적으로 어업활동을 할 수 있습니다. \n 이는 출원을 통해 국가로부터 부여되는 물권적 성격의 권리이며, 일정 수면을 점용할 수 있는 권리로 등록과 함께 어업권이 발생합니다.",contents_1,어업면허허가
5,어업허가,"어업허가 는 일정 기준을 충족하면 누구나 신청할 수 있으며, \n 수면의 독점 없이 공동 사용이 가능한 어업 방식입니다. \n 근해어업, 연안어업, 구획어업 등이 이에 해당하며 신청 조건에 따라 시장, 군수, 구청장 또는 해양수산부장관 등 행정기관이 허가를 부여합니다. \n 허가는 면허와 달리 법적 권리를 설정해주는 것이 아닌, 제한을 일시 해제하는 방식으로 일정한 범위 내에서 어업행위를 가능하게 합니다.",contents_1,어업면허허가
6,어업의 정의,"""어업""이란 국내 수산업법의 정의에 따르면, 수산동식물을 포획ㆍ채취하는 사업과 염전에서 바닷물을 자연 증발시켜 소금을 생산하는 사업을 말한다. ""수산업""이란 「수산업ㆍ어촌 발전 기본법」 제3조 제1호 각 목에 따른 어업ㆍ양식업ㆍ어획물운반업ㆍ수산물가공업 및 수산물유통업을 말한다. 이때 어업은 수산업에 해당한다.",contents_1,어업정의
7,귀어정책지원교육,"귀어 정책은 도시 거주자가 어촌으로 이주하여 어업에 종사할 수 있도록 돕는 정부 지원 제도입니다. 창업 자금, 주택 자금, 정착 지원금 등 다양한 재정적 지원을 받을 수 있으며, 신청 자격과 조건은 지역별 담당 기관을 통해 확인할 수 있습니다. 귀어 희망자는 '귀어학교'에서 4~6주간 이론과 실습 중심의 교육을 받으며 어촌 생활을 체험할 수 있습니다. 이 교육은 전국 6개 지역에서 무료로 운영되며, 어업 경험이 없는 초보자도 수강할 수 있습니다. 창업을 준비하는 경우 최대 3억 원의 창업 자금을 연 2%의 금리로 지원받을 수 있고, 주택을 구입하는 경우에는 최대 7,500만 원까지 지원받을 수 있습니다. 만 40세 미만의 청년 귀어인은 최대 3년간 매달 100만 원에서 80만 원까지의 정착 지원금을 받을 수 있습니",contents_2,귀어정책지원교육
8,귀어정책지원교육,"다. 이 외에도 만 49세 이하 어선 면허 보유 청년에게는 어선을 임대해주는 사업도 운영되고 있으며, 귀어인의 집을 통해 최대 2년간 어촌 생활을 미리 체험할 수 있는 임대주택도 제공됩니다. 모든 제도는 귀어인의 안정적 정착을 위한 것으로, 신청은 거주지 관할 시·군·구 또는 귀어귀촌 종합센터를 통해 진행할 수 있습니다.",contents_2,귀어정책지원교육
9,양식장 임대사업,"양식장 임대사업 2025년 양식장 임대 사업은 귀어·귀촌자 및 신규 양식업 진입자를 위한 정부 지원 제도입니다. 해양수산부 어촌양식정책과가 주관하고, 한국어촌어항공단 양식산업실이 실무를 맡아 운영합니다. 지원 대상은 청년 어업인, 귀어희망자, 수산업 창업 예정자 등이며, 임대 기간은 통상 3년 이내입니다. 사업계획서와 수산물 생산계획, 위생관리 기준 등을 제출하여 심사를 거쳐 선정됩니다. 신청은 관할 시·군·구 또는 한국어촌어항공단을 통해 가능하며, 관련 서류와 절차는 사업지침에 따라 운영됩니다.",contents_2,양식장및어선임대사업


In [19]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("chatbot/vector_store/contents")

In [ ]:

# file_path = mainPaths
# contentList = []

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,     # 총 길이
#     chunk_overlap=50,   # 겹치는 길이
#     separators=["\n\n", "다.", "요.", ".", "\n", " ", ""]
# )


# with open('./templates/contents/contents_5/지역별낚시산업.html', 'r', encoding='utf-8') as f:
#     html_content = f.read()

# folder = os.path.basename(os.path.dirname(i)) 
# page = os.path.splitext(os.path.basename(i))[0]

# soup = BeautifulSoup(html_content, 'html.parser')
# sections = soup.select('[name="section"]')

# for j in sections:
#     title = j.select('[name="content_title"]')
#     ths = j.select('table thead > tr > th')
#     storeNm = ths[0].text + " : "
#     storeUrl = ths[5].text + " : "
#     thsValue = j.select('table tbody tr')

#     print(title[0].text.strip())
#     for k in thsValue:
#         storeNmTxt = k.select('td')[0].text.strip()
#         storeUrlTxt = k.select('td')[5].text.strip()
#         print(storeNm + storeNmTxt)
#         print(storeUrl + storeUrlTxt)
#     print("=======================================")   


서울
상호명 : (사)한국스포츠피싱협회
홈페이지 : http://sportfishing.co.kr/
상호명 : (사)대한스포츠낚시중앙협회
홈페이지 : https://ksfca.modoo.at/
상호명 : 한국견지낚시협회
홈페이지 : http://www.gyeonji.net/
경기도
상호명 : (주)한국낚시채널
홈페이지 : https://www.eftv.co.kr/
상호명 : (사)한국낚시협회
홈페이지 : http://ekfa.kr
상호명 : 아오맥스
홈페이지 : http://www.aomex.co.kr
상호명 : 다미끼크라프트
홈페이지 : http://www.damiki.co.kr
상호명 : 배스랜드
홈페이지 : http://www.bassland.co.kr
상호명 : 런커
홈페이지 : http://www.lunkermall.com
상호명 : (사)한국낚시업중앙회
홈페이지 : https://www.koreafca.kr/
상호명 : 한국루어낚시협회
홈페이지 : http://www.lfa.or.kr
상호명 : 한국낚시방송
홈페이지 : http://www.fishingtv.kr/
인천
상호명 : 피싱조이
홈페이지 : http://www.fishingjoy.co.kr
상호명 : 에프마켓
홈페이지 : http://www.f-market.co.kr
부산
상호명 : 유승조구산업
홈페이지 : http://youseunghook.co.kr/
상호명 : 긱스
홈페이지 : http://gigskorea.com
상호명 : 동남보트 레저산업
홈페이지 : http://114boat.com
상호명 : (주)용성
홈페이지 : http://www.yong-sung.co.kr
대구·울산·경상도
상호명 : 놀림낚시
홈페이지 : http://nollim.co.kr
상호명 : (주)아피스
홈페이지 : http://www.apis.co.kr
상호명 : 코러낚시
홈페이지 : http://www.klfishing.com/
상호명 : 가자낚시
홈페이지 : http://www.ok1717.com
상호명 :